In [ ]:
from seqdata.core import *
from fastai2.basics import *
from fastai2.callback.progress import *

In [ ]:
#hide
%load_ext line_profiler

In [ ]:
hdf_files = get_hdf_files('test_data/')
src_df = df_source_items(hdf_files,[DfHDFCreateWindows(win_sz=1000+1,stp_sz=30,clm='current')])
src_df.head()

,path,l_slc,r_slc
0,test_data/train/Sim_RealisticCycle2.hdf5,0,1001
0,test_data/train/Sim_RealisticCycle2.hdf5,30,1031
0,test_data/train/Sim_RealisticCycle2.hdf5,60,1061
0,test_data/train/Sim_RealisticCycle2.hdf5,90,1091
0,test_data/train/Sim_RealisticCycle2.hdf5,120,1121


In [ ]:
tfms=[  [HDF2Sequence(['current','voltage']),SeqSlice(l_slc=1),toTensorSequencesInput],
        [HDF2Sequence(['voltage']),SeqSlice(r_slc=-1),toTensorSequencesOutput]]
dsrc = DataSource(src_df,tfms=tfms,splits=ParentSplitter()(src_df.path))

In [ ]:
# db = dsrc.databunch(bs=128,after_batch=[SeqNoiseInjection(std=[1.1,0.01]),Normalize(axes=[0,1])])
db = dsrc.databunch(bs=128,after_batch=[Cuda(),Normalize(axes=[0,1])])
db.one_batch()[0].shape

torch.Size([128, 1000, 2])

In [ ]:
class RecurrentNet(nn.Module):
    def __init__(self,input_size,output_size,num_layers=5,hidden_size=100):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.rnn = nn.GRU(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers,batch_first=True)
        self.final = nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # First layer
        out = x
        out,h = self.rnn(out)
#         print(out.shape)
        out = self.final(out)
#         print(out.shape)
#         import pdb; pdb.set_trace()
        return out

In [ ]:
lrn = Learner(db,RecurrentNet(2,1,num_layers=1),loss_func=nn.MSELoss())

In [ ]:
%%prun -s cumulative -l 20 -D batch.prof
lrn.fit(1)

epoch,train_loss,valid_loss,time
0,0.148762,0.008298,00:16


 
*** Profile stats marshalled to file 'batch.prof'. 


In [ ]:
%lprun?

In [ ]:
%%prun -s cumulative -l 20 -D batch.prof
db.one_batch()

 
*** Profile stats marshalled to file 'batch.prof'. 


In [ ]:
%%time
%lprun -f dsrc.tfms[1][0].init_enc -f dsrc.tls.__getitem__ db.one_batch()

CPU times: user 677 ms, sys: 5.59 ms, total: 682 ms
Wall time: 680 ms


In [ ]:
%lprun -f DataLoader.one_batch db.one_batch()

In [ ]:
%%timeit
dsrc[0]

UsageError: Cell magic `%%lprun` not found (But line magic `%lprun` exists, did you mean that instead?).


In [ ]:
#hide
from nbdev.export import *
notebook2script()